In [61]:
# pip install giotto-tda
# pip install gudhi

In [62]:
import pandas as pd
import numpy as np
from gtda.time_series import SingleTakensEmbedding, TakensEmbedding
import gudhi
import math
import cmath

series = np.log(pd.read_csv("SP500.csv", header=None).to_numpy().squeeze())

In [63]:
# initial values for important variables -- will have to find optimal values later on

n = 4 # dim of vectors
d = 5 # time delay
w = 5 # window size
epsilon = 0.02 # resolution threshold
q = 3 # number of precision qubits
simplex_dim = 3 # maximum simplex tree dimension used

In [64]:
def TakenEmbedding(time_series, dim, delay):
    serie = time_series.reshape(1, -1)
    STE = TakensEmbedding(time_delay=delay, dimension=dim)
    return STE.fit_transform(serie)[0]

In [65]:
def TakenPointCloud(time_series, dim, delay, window_size):
    embedding = TakenEmbedding(time_series, dim, delay)
    point_cloud = []
    l = len(embedding)
    for i in range(l - window_size):
        window = embedding[i : i + window_size]
        point_cloud.append(window)

    return np.array(point_cloud)

In [66]:
def SimplexTrees(clouds, simplices, e):
    tree = []

    for i in range(len(clouds)):
        rips_complex = gudhi.RipsComplex(points = clouds[i], max_edge_length = e)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension = simplices)

        simplex = [val for val, dist in simplex_tree.get_filtration()]

        simplex = sorted(simplex, key=lambda vector: (len(vector), vector))

        tree.append(simplex)

    return tree

In [67]:
def BoundaryOperators(simplex_tree, dimension):
    minus = [elem for elem in simplex_tree if len(elem) == dimension]
    forms = [elem for elem in simplex_tree if len(elem) == dimension + 1]

    if(dimension == 0):
        return np.array([[(-1) ** i] for i in range(len([item for item in simplex_tree if len(item) == 1]))])

    operator = []

    for simplex_minus in minus:
        simplices = []

        for simplex_form in forms:
            if(all(item in simplex_form for item in simplex_minus) and len(simplex_minus) + 1 == len(simplex_form)):
                val = list(set(simplex_form).symmetric_difference(simplex_minus))[0]
                if(val % 2 == 1):
                    simplices.append(-1)
                else:
                    simplices.append(1)
            else:
                simplices.append(0)

        operator.append(simplices)

    return np.array(operator)
# if we want each operator[i] to correspond to the ith simplex

In [68]:
point_clouds = TakenPointCloud(series, n, d, w)
# a numpy vector of numpy vectors of numpy vectors containing all the different point clouds.
print(point_clouds[0])
# point_cloud[t] gives the point cloud at time t

[[7.13389833 7.17081353 7.16110014 7.15205153]
 [7.14663437 7.1664011  7.14610857 7.14628387]
 [7.15787955 7.15850232 7.14739207 7.15252724]
 [7.15000317 7.15946102 7.14883305 7.16494271]
 [7.15460754 7.16732333 7.15964177 7.16181205]]


In [69]:
simplex_trees = SimplexTrees(point_clouds, simplex_dim, epsilon)
print(simplex_trees[0])
# simplex_trees[t] gives the simplex trees for the point cloud at time t

[[0], [1], [2], [3], [4], [1, 2], [2, 3], [2, 4], [3, 4], [2, 3, 4]]


In [75]:
temp = BoundaryOperators(simplex_trees[0], 1)
print(temp)
temp = BoundaryOperators(simplex_trees[0], 2)
print(temp)

[[ 0  0  0  0]
 [ 1  0  0  0]
 [-1 -1  1  0]
 [ 0  1  0  1]
 [ 0  0  1 -1]]
[[ 0]
 [ 1]
 [-1]
 [ 1]]


In [80]:
def CombinatorialLaplacian(tree, index):    
    delta_k = np.array(BoundaryOperators(tree, index))
    delta_l = np.array(BoundaryOperators(tree, index + 1))

    temp = np.matmul(delta_k.conj().T, delta_k) + np.matmul(delta_l, delta_l.conj().T)

    points = len(temp)
    dim = 2 ** math.ceil(math.log(len([items for items in tree if len(items) == index]), 2))
    pad = dim - points
    temp = np.pad(temp, (0, pad))

    gershgorin = []
    for i in range(len(temp)):
        sum = 0
        
        for j in range(len(temp[i])):
            if(i != j):
                sum += np.abs(temp[i][j])
        
        gershgorin.append(sum)

    max_eigenval = max(gershgorin)
    
    for i in range(points, dim):
        temp[i][i] = max_eigenval / 2
    
    return temp * 6 / max_eigenval # return val should be Hermitian

In [77]:
def unitarize(h):
    if not np.allclose(h, h.conj().T, atol = 1e-8):
        raise ValueError("Input matrix H must be Hermitian.")
    ih = 1j * h
    return np.exp(ih)

In [82]:
h = CombinatorialLaplacian(simplex_trees[0], 1)
print(h)
u = unitarize(h)
print(np.dot(u.conj().T, u))

[[ 2.4  1.2 -1.2  0.   0.   0.   0.   0. ]
 [ 1.2  3.6 -2.4  2.4  0.   0.   0.   0. ]
 [-1.2 -2.4  3.6 -2.4  0.   0.   0.   0. ]
 [ 0.   2.4 -2.4  3.6  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   2.4  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   2.4  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   2.4  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   2.4]]
[[8.        +0.00000000e+00j 3.24992808-5.13151811e-01j
  1.55658844-7.86586903e-01j 3.09056338-1.11798362e+00j
  3.24992808-1.11022302e-16j 3.24992808-1.11022302e-16j
  3.24992808-1.11022302e-16j 3.24992808-1.11022302e-16j]
 [3.24992808+5.13151811e-01j 8.        +0.00000000e+00j
  5.27044584+3.20701428e-01j 6.08707326-9.32039086e-01j
  0.25341819+1.85944538e-01j 0.25341819+1.85944538e-01j
  0.25341819+1.85944538e-01j 0.25341819+1.85944538e-01j]
 [1.55658844+7.86586903e-01j 5.27044584-3.20701428e-01j
  8.        +0.00000000e+00j 6.37019731-6.41255229e-02j
  0.25341819+3.40094907e+00j 0.25341819+3.40094907e+00j
  0.25341819+3.40094907e+00j 0.253418